In [1]:
import tarfile
import pandas as pd
import numpy as np
from nilearn import plotting
import os
import nibabel as nib
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(1)

C:\Users\zhang\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_info = pd.read_csv('data_info.csv')

In [3]:
data_info['Normal'] = data_info["diagnosis_0normal_1normaltomci_2mci_3ad_4otherdementia"].apply(lambda x: 1 if x==0 else 0)
data_info['NormalToMCI'] = data_info["diagnosis_0normal_1normaltomci_2mci_3ad_4otherdementia"].apply(lambda x: 1 if x==1 else 0)
data_info['MCI'] = data_info["diagnosis_0normal_1normaltomci_2mci_3ad_4otherdementia"].apply(lambda x: 1 if x==2 else 0)
data_info['AD'] = data_info["diagnosis_0normal_1normaltomci_2mci_3ad_4otherdementia"].apply(lambda x: 1 if x==3 else 0)
data_info['OtherDementia'] = data_info["diagnosis_0normal_1normaltomci_2mci_3ad_4otherdementia"].apply(lambda x: 1 if x==4 else 0)

In [4]:
data_info.head()

,Unnamed: 0,filename,NACCMRFI,diagnosis_0normal_1normaltomci_2mci_3ad_4otherdementia,Normal,NormalToMCI,MCI,AD,OtherDementia
0,0,mri71_MPRAGE_T1_AX_20100921133637_15.nii.gz,mri71,3.0,0,0,0,1,0
1,1,mri3579_t1_fl2d_sag_20100610142105_3.nii.gz,mri3579,3.0,0,0,0,1,0
2,2,mri3579_MPRAGE_adni1mm_ipat_20100610142105_11....,mri3579,3.0,0,0,0,1,0
3,3,mri3579_ep2d_t1w_128_2mm_20100610142105_19.nii.gz,mri3579,3.0,0,0,0,1,0
4,4,mri3579_MPRAGE_adni1mm_ipat_20100610142105_13....,mri3579,3.0,0,0,0,1,0


### select training set

In [5]:
number_files_loaded = 3
sample_list = data_info.iloc[:number_files_loaded, :]["filename"]
sample_data_list = list()
for filename in sample_list:
    # or is it better to use get_fdata()?
    a = nib.load("fs_t1/"+filename).get_data()
    sample_data_list.append(a)
sample_dataset = np.array(sample_data_list, dtype=np.float32)
batch_size, height, width, depth = sample_dataset.shape
channels = 1  # gray-scale instead of RGB
s = sample_dataset.reshape(number_files_loaded, 256, 256, 256, 1)

In [6]:
y = data_info.iloc[:number_files_loaded, 4:9]
y

,Normal,NormalToMCI,MCI,AD,OtherDementia
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0


### create placeholders

In [7]:
def create_placeholders(n_D0, n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.

    Arguments:
    n_D0 -- scalar, depth of an input image
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes

    Returns:
    X -- placeholder for the data input, of shape [None, n_D0, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    X = tf.placeholder(shape=(None, n_D0, n_H0, n_W0, n_C0), dtype=tf.float32)
    Y = tf.placeholder(shape=(None, n_y), dtype=tf.float32)

    return X, Y

### initialize parameters

### forward propagation

In [8]:
def forward_propagation(X, parameters=None):
    """
    Implements the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED

    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """

    # Retrieve the parameters from the dictionary "parameters"


    # CONV2D: filters W1, stride of 1, padding 'SAME', RELU
    # output_size = (3, 256, 256, 256, 8)
    A1 = tf.layers.conv3d(X, filters=8, kernel_size=4,
                          strides=1, padding="SAME", activation=tf.nn.relu)
    
    # MAXPOOL: window 8x8, sride 8, padding 'SAME'
    # output_size = (3, 32, 32, 32, 8)
    P1 = tf.layers.max_pooling3d(A1, pool_size=8, strides=8, padding="SAME")
    
    # CONV2D: filters W2, stride 1, padding 'SAME'
    # output_size = (3, 32, 32, 32, 16)
    A2 = tf.layers.conv3d(P1, filters=16, kernel_size=2,
                          strides=1, padding="SAME", activation=tf.nn.relu)
    
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    # output_size = (3, 8, 8, 8, 16)
    P2 = tf.layers.max_pooling3d(A2, pool_size=4, strides=4, padding="SAME") 
    
    # FLATTEN
    # output_size = (3, 8192)
    P2 = tf.contrib.layers.flatten(P2)   
    
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 5 neurons in output layer. Hint: one of the arguments should be "activation_fn=None"
    # output_size = (3,5)
    Z3 = tf.contrib.layers.fully_connected(P2, 5, activation_fn=None)   


    return Z3

### compute cost

In [9]:
def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    cost = tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y)
    cost = tf.reduce_mean(cost)
    
    return cost

In [10]:
(number_file_loaded, n_D0, n_H0, n_W0, n_C0) = s.shape
n_y = 5
with tf.Session() as sess:
    X, Y = create_placeholders(n_D0, n_H0, n_W0, n_C0, n_y)
#     parameters = initialize_parameters()
    Z3 = forward_propagation(X)
    cost = compute_cost(Z3, Y)
    init = tf.global_variables_initializer()
    sess.run(init)
    temp_cost = sess.run(cost, {X: s, Y: y})

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
temp_cost

55.668713